In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## שלב 1: הגדרת מודלים של Pydantic עבור פלטים מובנים

מודלים אלו מגדירים את **הסכמה** שהסוכנים יחזירו. שימוש ב-`response_format` עם Pydantic מבטיח:
- ✅ חילוץ נתונים בטוח מבחינת סוג
- ✅ אימות אוטומטי
- ✅ ללא שגיאות ניתוח מתגובות טקסט חופשי
- ✅ ניתוב מותנה קל בהתבסס על שדות


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## שלב 2: יצירת כלי להזמנת חדרים במלון

הכלי הזה הוא מה ש-**availability_agent** יקרא כדי לבדוק אם יש חדרים זמינים. אנו משתמשים במעטר `@ai_function` כדי:
- להמיר פונקציה בפייתון לכלי שניתן לקריאה על ידי AI
- ליצור באופן אוטומטי סכמת JSON עבור ה-LLM
- לטפל באימות פרמטרים
- לאפשר הפעלה אוטומטית על ידי סוכנים

לדוגמה זו:
- **סטוקהולם, סיאטל, טוקיו, לונדון, אמסטרדם** → יש חדרים ✅
- **כל הערים האחרות** → אין חדרים ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## שלב 3: הגדרת פונקציות תנאי לניתוב

פונקציות אלו בוחנות את תגובת הסוכן ומחליטות באיזו דרך לבחור בתהליך העבודה.

**תבנית מרכזית:**
1. בדוק אם ההודעה היא `AgentExecutorResponse`
2. נתח את הפלט המובנה (מודל Pydantic)
3. החזר `True` או `False` כדי לשלוט בנתיב הניתוב

תהליך העבודה יעריך את התנאים הללו על **קצוות** כדי להחליט איזה מבצע להפעיל בהמשך.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## שלב 4: יצירת מבצע תצוגה מותאם אישית

מבצעים הם רכיבי זרימת עבודה שמבצעים טרנספורמציות או השפעות צדדיות. אנו משתמשים במעטר `@executor` כדי ליצור מבצע מותאם אישית שמציג את התוצאה הסופית.

**מושגים מרכזיים:**
- `@executor(id="...")` - רושם פונקציה כמבצע זרימת עבודה
- `WorkflowContext[Never, str]` - רמזי סוג לקלט/פלט
- `ctx.yield_output(...)` - מניב את תוצאת זרימת העבודה הסופית


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## שלב 5: טעינת משתני סביבה

הגדר את לקוח ה-LLM. הדוגמה הזו מתאימה ל:
- **מודלים של GitHub** (רמת חינם עם טוקן GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## שלב 6: יצירת סוכני AI עם פלט מובנה

אנו יוצרים **שלושה סוכנים מתמחים**, כל אחד עטוף ב-`AgentExecutor`:

1. **availability_agent** - בודק זמינות חדרים במלון באמצעות הכלי
2. **alternative_agent** - מציע ערים חלופיות (כאשר אין חדרים זמינים)
3. **booking_agent** - מעודד הזמנה (כאשר יש חדרים זמינים)

**תכונות עיקריות:**
- `tools=[hotel_booking]` - מספק את הכלי לסוכן
- `response_format=PydanticModel` - מחייב פלט JSON מובנה
- `AgentExecutor(..., id="...")` - עוטף את הסוכן לשימוש בתהליך עבודה


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## שלב 7: בניית זרימת העבודה עם קצוות מותנים

עכשיו נשתמש ב-`WorkflowBuilder` כדי לבנות את הגרף עם ניתוב מותנה:

**מבנה זרימת העבודה:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**שיטות מרכזיות:**
- `.set_start_executor(...)` - מגדיר את נקודת הכניסה
- `.add_edge(from, to, condition=...)` - מוסיף קצה מותנה
- `.build()` - מסיים את בניית זרימת העבודה


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## שלב 8: הרצת מקרה בדיקה 1 - עיר ללא זמינות (פריז)

בואו נבדוק את המסלול של **ללא זמינות** על ידי בקשת מלונות בפריז (שאין בה חדרים בסימולציה שלנו).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## שלב 9: הרצת מקרה בדיקה 2 - עיר עם זמינות (סטוקהולם)

עכשיו נבדוק את מסלול **הזמינות** על ידי בקשת מלונות בסטוקהולם (שיש בה חדרים בסימולציה שלנו).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## נקודות מרכזיות ושלבים הבאים

### ✅ מה שלמדת:

1. **תבנית WorkflowBuilder**
   - השתמש ב-`.set_start_executor()` כדי להגדיר נקודת התחלה
   - השתמש ב-`.add_edge(from, to, condition=...)` לניתוב מותנה
   - קרא ל-`.build()` כדי לסיים את בניית זרימת העבודה

2. **ניתוב מותנה**
   - פונקציות תנאי בודקות את `AgentExecutorResponse`
   - מנתחות פלטים מובנים כדי לקבל החלטות ניתוב
   - מחזירות `True` כדי להפעיל חיבור, `False` כדי לדלג עליו

3. **שילוב כלים**
   - השתמש ב-`@ai_function` כדי להמיר פונקציות Python לכלי AI
   - סוכנים קוראים לכלים באופן אוטומטי כשצריך
   - כלים מחזירים JSON שסוכנים יכולים לנתח

4. **פלטים מובנים**
   - השתמש במודלים של Pydantic לחילוץ נתונים בטוח לפי סוג
   - הגדר `response_format=MyModel` בעת יצירת סוכנים
   - נתח תגובות עם `Model.model_validate_json()`

5. **מבצעים מותאמים אישית**
   - השתמש ב-`@executor(id="...")` כדי ליצור רכיבי זרימת עבודה
   - מבצעים יכולים לשנות נתונים או לבצע פעולות צד
   - השתמש ב-`ctx.yield_output()` כדי להפיק תוצאות זרימת עבודה

### 🚀 יישומים בעולם האמיתי:

- **הזמנת נסיעות**: בדיקת זמינות, הצעת חלופות, השוואת אפשרויות
- **שירות לקוחות**: ניתוב לפי סוג בעיה, תחושה, עדיפות
- **מסחר אלקטרוני**: בדיקת מלאי, הצעת חלופות, עיבוד הזמנות
- **מתן תוכן**: ניתוב לפי ציוני רעילות, דגלי משתמשים
- **אישורי תהליכים**: ניתוב לפי סכום, תפקיד משתמש, רמת סיכון
- **עיבוד רב-שלבי**: ניתוב לפי איכות נתונים, שלמות

### 📚 שלבים הבאים:

- הוסף תנאים מורכבים יותר (קריטריונים מרובים)
- יישם לולאות עם ניהול מצב זרימת עבודה
- הוסף תת-זרימות עבור רכיבים לשימוש חוזר
- שלב עם ממשקי API אמיתיים (הזמנת מלונות, מערכות מלאי)
- הוסף טיפול בשגיאות ונתיבי גיבוי
- הצג זרימות עבודה עם כלי ההדמיה המובנים



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
